In [2]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')
new_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/refs/heads/main/matrixes/new_matrix.csv')

In [3]:
venues

,Venue,Latitude,Longitude,Approx. Capacity,id
0,1932 Pool (Exposition Park),34.018060,-118.287500,100000,V1
1,Alamitos Beach,33.750000,-118.166670,10000,V2
2,Arena (Honda Center),33.806670,-117.867500,18000,V3
3,Arena (Crypto.com),34.043060,-118.267220,18910,V4
4,Arena (Intuit Dome),33.952780,-118.336110,17700,V5
5,Arena (Long Beach),33.751305,-118.188812,14000,V6
6,Belmont Shore,33.758890,-118.145000,10000,V7
7,Convention Center,34.043060,-118.267220,22870,V8
8,Convention Center,33.765830,-118.186940,13500,V9
9,Convention Center Lot,33.769720,-118.188890,55000,V10


# SUMMARY OF DATA COLLECTION & FLP

In [41]:
venues = venues.dropna()

In [42]:
import folium

# Create a map centered at the average latitude and longitude
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add markers for each cluster
for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m)


# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m)
    
# Add markers for each bus terminal
for _, row in bus_terminals.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='green')
    ).add_to(m)
# Display the map
m

In [43]:
demand= accomodations_clusters['total_accommodates'].to_numpy()
demand

array([ 6969,  6395,  2801,  7026,  4909,  1545,  6342,  6399,  1171,
        4117,  3110,  3088,  5080,  4779,  2666,  9095,  7555,  4679,
        2111,  3376,  7538,  7068,  6853,  1576,  2974,  2110,  1319,
       10383,  1268,   273,  8063,  8239,  3833,  6408,  7028,  2057,
        4099,  2060,  2646,  2505,  6751,   934,  4119,  9229,   563,
        7296,  2670,  2156,  2263,  1619, 12812])

In [44]:
from ortools.sat.python import cp_model

# Set the number of bus stops to locate
p = 15

# bus_matrix rows: accommodations; columns: candidate bus terminals
accom = list(bus_matrix.index)
bus_stops = list(bus_matrix.columns[1:])

model = cp_model.CpModel()

# Decision variables:
x = {j: model.NewBoolVar(f"x_{j}") for j in bus_stops}
y = {}
for idx, i in enumerate(accom):
    for j in bus_stops:
        y[(idx, j)] = model.NewBoolVar(f"y_{idx}_{j}")

# Constraint: each accommodation must be assigned to exactly one bus stop.
for idx, i in enumerate(accom):
    model.Add(sum(y[(idx, j)] for j in bus_stops) == 1)

# Constraint: assignment only possible if bus stop is selected.
for idx, i in enumerate(accom):
    for j in bus_stops:
        model.Add(y[(idx, j)] <= x[j])

# Constraint: exactly p bus stops are selected.
model.Add(sum(x[j] for j in bus_stops) == p)

# Objective: minimize total (demand-weighted) distance.
# We convert bus_matrix.loc[i, j] to an integer cost.
objective_terms = []
for idx, i in enumerate(accom):
    for j in bus_stops:
        # Multiply demand and the scaled distance.
        cost = int(round(bus_matrix.loc[i, j]))
        objective_terms.append(demand[idx] * cost * y[(idx, j)])
model.Minimize(sum(objective_terms))

# Solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    selected = [j for j in bus_stops if solver.Value(x[j]) == 1]
    print("Selected bus stops:", selected)
else:
    print("No solution found.")

Selected bus stops: ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']


In [45]:
#map only selected bus stops ['BD14', 'BD15', 'BT03', 'BL14', 'BT23', 'BT18', 'BL23', 'BL07', 'BT05']
# Add markers for each cluster
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m2 = folium.Map(location=map_center, zoom_start=10)

for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m2)
# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m2)

for _, row in bus_terminals[bus_terminals['id'].isin(selected)].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='lightgreen', icon='ok-sign')
    ).add_to(m2)
# Display the map with selected bus stops
m2

In [46]:
merged_matrix.set_index('Unnamed: 0', inplace=True)
merged_matrix

,A1,A10,A11,A12,A13,A14,A15,A16,A17,A18,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A1,0.0,3304.0,3496.0,1727.0,1956.0,2020.0,2410.0,1512.0,3273.0,3098.0,...,1850.0,2149.0,1860.0,2759.0,2024.0,2357.0,2234.0,1594.0,1485.0,1745.0
A10,3263.0,0.0,4901.0,2745.0,2974.0,3033.0,2460.0,2917.0,2778.0,3209.0,...,1696.0,1627.0,2491.0,2459.0,2890.0,2960.0,2575.0,2406.0,2420.0,2006.0
A11,3459.0,4958.0,0.0,3305.0,3534.0,3598.0,4079.0,2432.0,4852.0,4676.0,...,3477.0,3757.0,3064.0,4313.0,3578.0,3911.0,3788.0,3326.0,3408.0,3477.0
A12,1753.0,2784.0,3117.0,0.0,593.0,1014.0,1617.0,1405.0,2390.0,2214.0,...,1556.0,1785.0,2037.0,1875.0,1140.0,1473.0,1350.0,1783.0,1424.0,1630.0
A13,1909.0,2939.0,3272.0,576.0,0.0,489.0,1654.0,1561.0,2426.0,2250.0,...,1711.0,1940.0,2193.0,1911.0,1176.0,1509.0,1386.0,1938.0,1579.0,1785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,1326.0,...,1847.0,1958.0,2368.0,1327.0,980.0,0.0,934.0,1965.0,1990.0,1634.0
BT25,2234.0,2575.0,3788.0,1350.0,1386.0,1332.0,250.0,1918.0,1442.0,1234.0,...,1126.0,1238.0,1647.0,840.0,857.0,952.0,0.0,1244.0,1479.0,914.0
BT26,1594.0,2406.0,3326.0,1783.0,1938.0,2233.0,1496.0,1365.0,2523.0,2315.0,...,933.0,1044.0,1418.0,1854.0,2018.0,2087.0,1345.0,0.0,357.0,707.0


In [47]:
print("Number of NaN values in merged_matrix:", merged_matrix.isna().sum().sum())

Number of NaN values in merged_matrix: 610


In [48]:
print("Rows with NaNs:")
print(merged_matrix[merged_matrix.isna().any(axis=1)])

print("Columns with NaNs:")
print(merged_matrix.columns[merged_matrix.isna().any()])


Rows with NaNs:
                A1     A10     A11     A12     A13     A14     A15     A16  \
Unnamed: 0                                                                   
A1             0.0  3304.0  3496.0  1727.0  1956.0  2020.0  2410.0  1512.0   
A10         3263.0     0.0  4901.0  2745.0  2974.0  3033.0  2460.0  2917.0   
A11         3459.0  4958.0     0.0  3305.0  3534.0  3598.0  4079.0  2432.0   
A12         1753.0  2784.0  3117.0     0.0   593.0  1014.0  1617.0  1405.0   
A13         1909.0  2939.0  3272.0   576.0     0.0   489.0  1654.0  1561.0   
...            ...     ...     ...     ...     ...     ...     ...     ...   
BT24        2357.0  2960.0  3911.0  1473.0  1509.0  1455.0  1027.0  2040.0   
BT25        2234.0  2575.0  3788.0  1350.0  1386.0  1332.0   250.0  1918.0   
BT26        1594.0  2406.0  3326.0  1783.0  1938.0  2233.0  1496.0  1365.0   
BT27        1485.0  2420.0  3408.0  1424.0  1579.0  1874.0  1556.0  1447.0   
BT28        1745.0  2006.0  3477.0  1630.0  1785

In [49]:
nan_positions = merged_matrix.isna()
print("NaN symmetric check (should be True for all):")
print((nan_positions == nan_positions.T).all())


NaN symmetric check (should be True for all):
A1      True
A10     True
A11     True
A12     True
A13     True
        ... 
BT24    True
BT25    True
BT26    True
BT27    True
BT28    True
Length: 154, dtype: bool


In [50]:
max_dist = merged_matrix.max().max()
fill_value = max_dist * 10
merged_matrix = merged_matrix.fillna(fill_value)
print("Filled merged_matrix:")
print(merged_matrix)

Filled merged_matrix:
                A1     A10     A11     A12     A13     A14     A15     A16  \
Unnamed: 0                                                                   
A1             0.0  3304.0  3496.0  1727.0  1956.0  2020.0  2410.0  1512.0   
A10         3263.0     0.0  4901.0  2745.0  2974.0  3033.0  2460.0  2917.0   
A11         3459.0  4958.0     0.0  3305.0  3534.0  3598.0  4079.0  2432.0   
A12         1753.0  2784.0  3117.0     0.0   593.0  1014.0  1617.0  1405.0   
A13         1909.0  2939.0  3272.0   576.0     0.0   489.0  1654.0  1561.0   
...            ...     ...     ...     ...     ...     ...     ...     ...   
BT24        2357.0  2960.0  3911.0  1473.0  1509.0  1455.0  1027.0  2040.0   
BT25        2234.0  2575.0  3788.0  1350.0  1386.0  1332.0   250.0  1918.0   
BT26        1594.0  2406.0  3326.0  1783.0  1938.0  2233.0  1496.0  1365.0   
BT27        1485.0  2420.0  3408.0  1424.0  1579.0  1874.0  1556.0  1447.0   
BT28        1745.0  2006.0  3477.0  1630.0

In [51]:
print("Highest value in merged_matrix:", merged_matrix.values.max())

Highest value in merged_matrix: 718410.0


In [52]:
# Extract rows corresponding to the selected bus stops
selected_bus_stops_matrix = merged_matrix.loc[selected]
selected_bus_stops_matrix

,A1,A10,A11,A12,A13,A14,A15,A16,A17,A18,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
BD14,2089.0,3107.0,3644.0,749.0,260.0,669.0,1754.0,1773.0,2633.0,2435.0,...,1851.0,1994.0,2391.0,2072.0,1279.0,1691.0,1572.0,1856.0,1727.0,1670.0
BD15,452.0,3066.0,3726.0,1556.0,1711.0,2006.0,2202.0,1765.0,3122.0,2924.0,...,1810.0,1953.0,2151.0,2560.0,1768.0,2180.0,2051.0,1553.0,1186.0,1629.0
BD16,2703.0,3916.0,2465.0,2362.0,2517.0,2814.0,3051.0,1624.0,3929.0,3732.0,...,2386.0,2671.0,2058.0,3369.0,2576.0,2987.0,2869.0,2415.0,2474.0,2364.0
BT16,2149.0,1620.0,3757.0,1666.0,1822.0,2116.0,1413.0,1796.0,1999.0,2208.0,...,431.0,708.0,1267.0,1470.0,1971.0,1979.0,1298.0,1153.0,1474.0,679.0
BT03,1878.0,3284.0,2537.0,1471.0,1627.0,1924.0,2160.0,624.0,3039.0,2842.0,...,1753.0,1942.0,1417.0,2479.0,1685.0,2097.0,1979.0,1685.0,1744.0,1666.0
BT08,1788.0,2801.0,3342.0,904.0,927.0,873.0,972.0,1471.0,1851.0,1653.0,...,1545.0,1688.0,2089.0,1290.0,497.0,909.0,790.0,1550.0,1421.0,1364.0
BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,1326.0,...,1847.0,1958.0,2368.0,1327.0,980.0,0.0,934.0,1965.0,1990.0,1634.0
BL14,2279.0,2498.0,3992.0,1553.0,1590.0,1536.0,325.0,2210.0,1235.0,1027.0,...,1164.0,1276.0,1685.0,709.0,1061.0,798.0,251.0,1282.0,1517.0,952.0
BD04,3152.0,2792.0,4706.0,2268.0,2304.0,2250.0,1349.0,2836.0,666.0,1403.0,...,1808.0,1934.0,2558.0,1297.0,1775.0,1387.0,1255.0,2286.0,2521.0,1909.0


In [53]:
# Extract the indices of the selected bus stops from the merged matrix
selected_indices = merged_matrix.index.get_indexer(selected_bus_stops_matrix.index)
selected_indices

array([ 98,  99, 100, 141, 128, 133, 149, 115,  88, 136, 131, 146, 121,
       143, 124])

In [54]:
# Create a list of pickup and delivery pairs
pickup_delivery_pairs = [(accomodation_id, venue_id) for accomodation_id in accomodations_clusters['id'] for venue_id in venues['id']]

# Display the list of pairs
pickup_delivery_pairs
pickup_delivery_pairs = pickup_delivery_pairs[:5]  # test with 10 requests


# Coordinates dataframe

# VRP with merged_matrix

In [55]:
#import pandas as pd

# Your selected bus stops:
#selected_stops = ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24',
                  #'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']

# Assume merged_matrix is your original big DataFrame

# Filter rows and columns to keep only selected stops
#reduced_matrix = merged_matrix.loc[selected_stops, selected_stops].copy()

In [56]:
##max_dist = reduced_matrix.max().max()
#fill_value = max_dist * 10
#reduced_matrix = reduced_matrix.fillna(fill_value)
#print("Filled merged_matrix:")
#print(reduced_matrix)

In [57]:
#import pandas as pd

# Replace 'your_username' with your actual username or provide the full path to the file
#new_matrix_path = '/Users/your_username/Downloads/new_matrix.csv'

# Load the CSV file into a DataFrame
#new_matrix = pd.read_csv(new_matrix_path)

# Display the first few rows of the DataFrame
#new_matrix.head()

In [58]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import numpy as np

In [59]:
import pandas as pd

# List of selected bus stops
selected_stops = ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 
                  'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']

# Suppose `merged_matrix` is a pandas DataFrame where rows and columns are stop names
# You want to keep only the selected rows and columns

filtered_matrix = merged_matrix.loc[selected_stops, selected_stops].copy()

In [60]:
def create_data_model():
    data = {}
    data['distance_matrix'] = merged_matrix.values.tolist()
    data['num_vehicles'] = 5
    
    # These are the actual node indices in the full matrix (from FLP)
    data['starts'] = [98, 99, 100, 141, 128]
    data['ends'] = data['starts']  # if routes must return to starting depot
    
    data['pickups_deliveries'] = pickup_delivery_pairs
    data['venues'] = set(venues)
    
    return data

data = create_data_model()


In [61]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['starts'], data['ends'])
routing = pywrapcp.RoutingModel(manager)

In [62]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    if from_index < 0 or from_index >= routing.Size():
        print(f"Invalid from_index: {from_index} (routing size {routing.Size()})")
        raise ValueError("from_index out of range")
    if to_index < 0 or to_index >= routing.Size():
        print(f"Invalid to_index: {to_index} (routing size {routing.Size()})")
        raise ValueError("to_index out of range")

    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)

    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [63]:
print(f"Number of nodes (manager): {manager.GetNumberOfNodes()}")
print(f"Routing model size: {routing.Size()}")
print(f"Distance matrix size: {len(data['distance_matrix'])}")


Number of nodes (manager): 154
Routing model size: 154
Distance matrix size: 154


In [64]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [65]:
# Create a mapping for non-integer identifiers
node_mapping = {node: idx for idx, node in enumerate(set(sum(pickup_delivery_pairs, ())))}

# Convert pickup_delivery_pairs to use integer indices
data['pickups_deliveries'] = [
    (node_mapping[pickup], node_mapping[delivery]) for pickup, delivery in pickup_delivery_pairs
]

# Add pickup and delivery constraints
for pickup, delivery in data['pickups_deliveries']:
    pickup_index = manager.NodeToIndex(pickup)
    delivery_index = manager.NodeToIndex(delivery)
    routing.AddPickupAndDelivery(pickup_index, delivery_index)

In [66]:
print(f"Routing size: {routing.Size()}")
print(f"Number of nodes (manager): {manager.GetNumberOfNodes()}")
print(f"Distance matrix size: {len(data['distance_matrix'])} x {len(data['distance_matrix'][0])}")
print(f"Vehicle start indices: {data['starts']}")
print(f"Vehicle end indices: {data['ends']}")


Routing size: 154
Number of nodes (manager): 154
Distance matrix size: 154 x 154
Vehicle start indices: [98, 99, 100, 141, 128]
Vehicle end indices: [98, 99, 100, 141, 128]


In [67]:
#Ensure all nodes are visited
penalty = 100000  # high enough to force consideration
for node in range(1, len(data['distance_matrix'])):
    if node not in data['starts'] and node not in data['ends']:
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

In [68]:
#limit each route to do max 25 stops
routing.AddConstantDimension(
    1,  # Increment by 1 per stop
    25,  # Maximum number of stops per vehicle
    True,  # Start cumul at zero
    'StopCount'
)

# (Optional) Get the dimension to enforce additional constraints if needed
stop_count_dimension = routing.GetDimensionOrDie('StopCount')

In [ ]:
#solver
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.SAVINGS)

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

search_parameters.time_limit.seconds = 30

solution = routing.SolveWithParameters(search_parameters)

#print solution
def print_solution(data, manager, routing, solution):
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            plan_output += f' {node} ->'
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        node = manager.IndexToNode(index)
        plan_output += f' {node}\nDistance of the route: {route_distance}m\n'
        print(plan_output)
        total_distance += route_distance
    print(f'Total distance of all routes: {total_distance}m')

if solution:
    print_solution(data, manager, routing, solution)
else:
    print("No solution found.")


In [ ]:
# Create a map centered at the average latitude and longitude of the venues
map_center = [venues['Latitude'].mean(), venues['Longitude'].mean()]
route_map = folium.Map(location=map_center, zoom_start=10)

# Add smaller markers for each venue
for _, row in venues.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,  # Smaller radius for the marker
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=f"Venue: {row['Venue']}<br>ID: {row['id']}<br>Capacity: {row['Approx. Capacity']}"
    ).add_to(route_map)

# Add smaller markers for each selected bus stop
for stop_id in selected_stops:
    stop_row = bus_terminals[bus_terminals['id'] == stop_id].iloc[0]
    folium.CircleMarker(
        location=[stop_row['Latitude'], stop_row['Longitude']],
        radius=5,  # Smaller radius for the marker
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        popup=f"Bus Stop: {stop_row['FACILITY']}<br>ID: {stop_row['id']}"
    ).add_to(route_map)

# Draw routes for each vehicle
colors = ['red', 'purple', 'orange', 'darkblue', 'darkgreen', 'brown', 'pink']  # Colors for routes
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    route_coordinates = []
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)  # Get the actual node index
        if node in selected_stops:  # Check if the node is in selected_stops
            stop_id = node  # Node index corresponds to the stop ID
            stop_row = bus_terminals[bus_terminals['id'] == stop_id].iloc[0]
            route_coordinates.append([stop_row['Latitude'], stop_row['Longitude']])
        index = solution.Value(routing.NextVar(index))
    
    # Add the route to the map if it has coordinates
    if route_coordinates:
        folium.PolyLine(
            route_coordinates,
            color=colors[vehicle_id % len(colors)],  # Cycle through colors
            weight=5,
            opacity=0.8
        ).add_to(route_map)
    else:
        print(f"Vehicle {vehicle_id} has no assigned route.")

# Display the map
route_map

AttributeError: 'NoneType' object has no attribute 'Value'

# VRP with reduced matrix

In [4]:
new_matrix

,Unnamed: 0,BD14,BD15,BD16,BT16,BT03,BT08,BT24,BL14,BD04,...,A42,A43,A44,A45,A46,A47,A48,A49,A50,A51
0,BD14,0.0,1947.0,2868.0,1976.0,1858.0,1052.0,1691.0,1614.0,2546.0,...,4028.0,1868.0,2199.0,2731.0,1740.0,2078.0,1859.0,1696.0,1981.0,1607.0
1,BD15,1769.0,0.0,2951.0,1936.0,2129.0,1551.0,2180.0,2062.0,3054.0,...,4130.0,1827.0,1880.0,2813.0,1699.0,1621.0,1334.0,1592.0,1788.0,1566.0
2,BD16,2574.0,2932.0,0.0,2681.0,1420.0,2359.0,2987.0,2910.0,3843.0,...,3226.0,2700.0,1826.0,1577.0,2519.0,2383.0,2482.0,2661.0,2490.0,2726.0
3,BT16,1879.0,2037.0,2695.0,0.0,2130.0,1661.0,1979.0,1309.0,1598.0,...,4005.0,215.0,1603.0,2844.0,647.0,1070.0,1250.0,974.0,978.0,871.0
4,BT03,1684.0,2044.0,1693.0,2049.0,0.0,1469.0,2097.0,2020.0,2953.0,...,2850.0,2067.0,1163.0,1624.0,1694.0,1558.0,1657.0,1836.0,1666.0,1873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,A47,2078.0,1621.0,2383.0,1070.0,1558.0,1772.0,2042.0,1360.0,2299.0,...,3537.0,1007.0,1228.0,2259.0,687.0,0.0,722.0,795.0,564.0,858.0
76,A48,1859.0,1334.0,2482.0,1250.0,1657.0,1553.0,2115.0,1432.0,2436.0,...,3637.0,1187.0,1328.0,2359.0,883.0,705.0,0.0,458.0,969.0,831.0
77,A49,1696.0,1592.0,2661.0,974.0,1836.0,1390.0,1669.0,987.0,1991.0,...,3816.0,874.0,1507.0,2538.0,592.0,719.0,458.0,0.0,889.0,396.0
78,A50,1981.0,1788.0,2490.0,978.0,1666.0,1675.0,1945.0,1263.0,2207.0,...,3644.0,915.0,1255.0,2367.0,595.0,560.0,916.0,840.0,0.0,766.0


In [5]:
new_matrix.set_index('Unnamed: 0', inplace=True)
new_matrix

,BD14,BD15,BD16,BT16,BT03,BT08,BT24,BL14,BD04,BT11,...,A42,A43,A44,A45,A46,A47,A48,A49,A50,A51
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
BD14,0.0,1947.0,2868.0,1976.0,1858.0,1052.0,1691.0,1614.0,2546.0,1472.0,...,4028.0,1868.0,2199.0,2731.0,1740.0,2078.0,1859.0,1696.0,1981.0,1607.0
BD15,1769.0,0.0,2951.0,1936.0,2129.0,1551.0,2180.0,2062.0,3054.0,1431.0,...,4130.0,1827.0,1880.0,2813.0,1699.0,1621.0,1334.0,1592.0,1788.0,1566.0
BD16,2574.0,2932.0,0.0,2681.0,1420.0,2359.0,2987.0,2910.0,3843.0,2627.0,...,3226.0,2700.0,1826.0,1577.0,2519.0,2383.0,2482.0,2661.0,2490.0,2726.0
BT16,1879.0,2037.0,2695.0,0.0,2130.0,1661.0,1979.0,1309.0,1598.0,1154.0,...,4005.0,215.0,1603.0,2844.0,647.0,1070.0,1250.0,974.0,978.0,871.0
BT03,1684.0,2044.0,1693.0,2049.0,0.0,1469.0,2097.0,2020.0,2953.0,1737.0,...,2850.0,2067.0,1163.0,1624.0,1694.0,1558.0,1657.0,1836.0,1666.0,1873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A47,2078.0,1621.0,2383.0,1070.0,1558.0,1772.0,2042.0,1360.0,2299.0,1219.0,...,3537.0,1007.0,1228.0,2259.0,687.0,0.0,722.0,795.0,564.0,858.0
A48,1859.0,1334.0,2482.0,1250.0,1657.0,1553.0,2115.0,1432.0,2436.0,1002.0,...,3637.0,1187.0,1328.0,2359.0,883.0,705.0,0.0,458.0,969.0,831.0
A49,1696.0,1592.0,2661.0,974.0,1836.0,1390.0,1669.0,987.0,1991.0,836.0,...,3816.0,874.0,1507.0,2538.0,592.0,719.0,458.0,0.0,889.0,396.0


In [6]:
# Check if the matrix is square
assert new_matrix.shape[0] == new_matrix.shape[1], "Distance matrix must be square"

# Check if all values are numeric and non-null
assert new_matrix.notnull().all().all(), "Distance matrix contains NaN values"
assert (new_matrix.dtypes == float).all(), "Distance matrix must contain only floats"
assert list(new_matrix.index) == list(new_matrix.columns), "Row and column labels must match"

new_matrix = new_matrix.loc[new_matrix.columns]
assert new_matrix.shape[0] == new_matrix.shape[1]
assert new_matrix.notnull().all().all()

In [7]:
new_matrix

,BD14,BD15,BD16,BT16,BT03,BT08,BT24,BL14,BD04,BT11,...,A42,A43,A44,A45,A46,A47,A48,A49,A50,A51
BD14,0.0,1947.0,2868.0,1976.0,1858.0,1052.0,1691.0,1614.0,2546.0,1472.0,...,4028.0,1868.0,2199.0,2731.0,1740.0,2078.0,1859.0,1696.0,1981.0,1607.0
BD15,1769.0,0.0,2951.0,1936.0,2129.0,1551.0,2180.0,2062.0,3054.0,1431.0,...,4130.0,1827.0,1880.0,2813.0,1699.0,1621.0,1334.0,1592.0,1788.0,1566.0
BD16,2574.0,2932.0,0.0,2681.0,1420.0,2359.0,2987.0,2910.0,3843.0,2627.0,...,3226.0,2700.0,1826.0,1577.0,2519.0,2383.0,2482.0,2661.0,2490.0,2726.0
BT16,1879.0,2037.0,2695.0,0.0,2130.0,1661.0,1979.0,1309.0,1598.0,1154.0,...,4005.0,215.0,1603.0,2844.0,647.0,1070.0,1250.0,974.0,978.0,871.0
BT03,1684.0,2044.0,1693.0,2049.0,0.0,1469.0,2097.0,2020.0,2953.0,1737.0,...,2850.0,2067.0,1163.0,1624.0,1694.0,1558.0,1657.0,1836.0,1666.0,1873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A47,2078.0,1621.0,2383.0,1070.0,1558.0,1772.0,2042.0,1360.0,2299.0,1219.0,...,3537.0,1007.0,1228.0,2259.0,687.0,0.0,722.0,795.0,564.0,858.0
A48,1859.0,1334.0,2482.0,1250.0,1657.0,1553.0,2115.0,1432.0,2436.0,1002.0,...,3637.0,1187.0,1328.0,2359.0,883.0,705.0,0.0,458.0,969.0,831.0
A49,1696.0,1592.0,2661.0,974.0,1836.0,1390.0,1669.0,987.0,1991.0,836.0,...,3816.0,874.0,1507.0,2538.0,592.0,719.0,458.0,0.0,889.0,396.0
A50,1981.0,1788.0,2490.0,978.0,1666.0,1675.0,1945.0,1263.0,2207.0,1122.0,...,3644.0,915.0,1255.0,2367.0,595.0,560.0,916.0,840.0,0.0,766.0


In [8]:
print("Index of new_matrix:", new_matrix.index)
print("Columns of new_matrix:", new_matrix.columns)

Index of new_matrix: Index(['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 'BL14', 'BD04',
       'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23', 'V9', 'V30', 'V5',
       'V31', 'V17', 'V14', 'V24', 'V8', 'V11', 'V12', 'V19', 'V18', 'V28',
       'V20', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20',
       'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30',
       'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40',
       'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50',
       'A51'],
      dtype='object')
Columns of new_matrix: Index(['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 'BL14', 'BD04',
       'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23', 'V9', 'V30', 'V5',
       'V31', 'V17', 'V14', 'V24', 'V8', 'V11', 'V12', 'V19', 'V18', 'V28',
       'V20', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
   

In [9]:
# Extract venue and accommodation IDs from the new_matrix index and columns
venues_ids = [col for col in new_matrix.columns if col.startswith('V')]
accommodations_ids = [col for col in new_matrix.columns if col.startswith('A')]

# Create a list of pairs between venues and accommodations
venue_accommodation_pairs = [(venue, accommodation) for venue in venues_ids for accommodation in accommodations_ids]

# Display the list of pairs
venue_accommodation_pairs

[('V9', 'A1'),
 ('V9', 'A2'),
 ('V9', 'A3'),
 ('V9', 'A4'),
 ('V9', 'A5'),
 ('V9', 'A6'),
 ('V9', 'A7'),
 ('V9', 'A8'),
 ('V9', 'A9'),
 ('V9', 'A10'),
 ('V9', 'A11'),
 ('V9', 'A12'),
 ('V9', 'A13'),
 ('V9', 'A14'),
 ('V9', 'A15'),
 ('V9', 'A16'),
 ('V9', 'A17'),
 ('V9', 'A18'),
 ('V9', 'A19'),
 ('V9', 'A20'),
 ('V9', 'A21'),
 ('V9', 'A22'),
 ('V9', 'A23'),
 ('V9', 'A24'),
 ('V9', 'A25'),
 ('V9', 'A26'),
 ('V9', 'A27'),
 ('V9', 'A28'),
 ('V9', 'A29'),
 ('V9', 'A30'),
 ('V9', 'A31'),
 ('V9', 'A32'),
 ('V9', 'A33'),
 ('V9', 'A34'),
 ('V9', 'A35'),
 ('V9', 'A36'),
 ('V9', 'A37'),
 ('V9', 'A38'),
 ('V9', 'A39'),
 ('V9', 'A40'),
 ('V9', 'A41'),
 ('V9', 'A42'),
 ('V9', 'A43'),
 ('V9', 'A44'),
 ('V9', 'A45'),
 ('V9', 'A46'),
 ('V9', 'A47'),
 ('V9', 'A48'),
 ('V9', 'A49'),
 ('V9', 'A50'),
 ('V9', 'A51'),
 ('V30', 'A1'),
 ('V30', 'A2'),
 ('V30', 'A3'),
 ('V30', 'A4'),
 ('V30', 'A5'),
 ('V30', 'A6'),
 ('V30', 'A7'),
 ('V30', 'A8'),
 ('V30', 'A9'),
 ('V30', 'A10'),
 ('V30', 'A11'),
 ('V30', 'A12')

In [10]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2


def create_data_model():
    data = {}
    data['distance_matrix'] = new_matrix.values.tolist()
    data['num_vehicles'] = 10
    
    # Convert the first 5 rows of new_matrix to integer indices
    data['starts'] = list(range(10))  # First 5 rows correspond to indices 0, 1, 2, 3, 4
    data['ends'] = data['starts']  # Routes must return to starting depot
    
    data['pickups_deliveries'] = venue_accommodation_pairs
    data['venues'] = set(venues)
    
    return data

data = create_data_model()

In [11]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['starts'], data['ends'])
routing = pywrapcp.RoutingModel(manager)

In [12]:
def distance_callback(from_index, to_index):
    from_index = int(from_index)
    to_index = int(to_index)
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [13]:
# Add fixed cost to encourage fewer vehicles
fixed_cost = 10000  # adjust to tune tradeoff between #vehicles and total distance
for v in range(data['num_vehicles']):
    routing.SetFixedCostOfVehicle(fixed_cost, v)

In [14]:
#Ensure all nodes are visited
#penalty = 100000  # high enough to force consideration
#for node in range(1, len(data['distance_matrix'])):
    #if node not in data['starts'] and node not in data['ends']:
        #routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

In [15]:
#limit each route to do max 10 stops
routing.AddConstantDimension(
    1,  # Increment by 1 per stop
    10,  # Maximum number of stops per vehicle
    True,  # Start cumul at zero
    'StopCount'
)

# (Optional) Get the dimension to enforce additional constraints if needed
stop_count_dimension = routing.GetDimensionOrDie('StopCount')


In [16]:
#limit maximum distance per route
max_distance = 30000  # e.g. 30 km
for vehicle_id in range(data['num_vehicles']):
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        max_distance,
        True,
        'Distance')


In [17]:
print("Distance Matrix Shape:", len(data['distance_matrix']), len(data['distance_matrix'][0]))
print("Distance Matrix Example Row:", data['distance_matrix'][0])

Distance Matrix Shape: 80 80
Distance Matrix Example Row: [0.0, 1947.0, 2868.0, 1976.0, 1858.0, 1052.0, 1691.0, 1614.0, 2546.0, 1472.0, 1358.0, 2391.0, 1639.0, 2318.0, 2531.0, 2670.0, 404.0, 2802.0, 2633.0, 2449.0, 2974.0, 1555.0, 2647.0, 3447.0, 1835.0, 1592.0, 3029.0, 1756.0, 2358.0, 2089.0, 1610.0, 1216.0, 1703.0, 2954.0, 3293.0, 2245.0, 1968.0, 2966.0, 3107.0, 3644.0, 749.0, 260.0, 669.0, 1754.0, 1773.0, 2633.0, 2435.0, 2346.0, 2221.0, 2139.0, 2109.0, 2143.0, 2979.0, 3202.0, 1567.0, 1359.0, 1437.0, 1673.0, 5117.0, 1531.0, 2352.0, 1121.0, 1241.0, 2099.0, 3537.0, 2540.0, 4995.0, 2986.0, 2243.0, 1062.0, 4028.0, 1868.0, 2199.0, 2731.0, 1740.0, 2078.0, 1859.0, 1696.0, 1981.0, 1607.0]


In [18]:
max_index = len(data['distance_matrix']) - 1
for node in data['starts'] + data['ends']:
    if node > max_index:
        print(f"Invalid node index: {node}")

In [19]:
#solver
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.SAVINGS)

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

search_parameters.time_limit.seconds = 30

solution = routing.SolveWithParameters(search_parameters)

def count_used_routes(routing, manager, solution):
    used = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        if not routing.IsEnd(solution.Value(routing.NextVar(index))):
            used += 1
    return used

actual_vehicles_used = count_used_routes(routing, manager, solution)
print(f"Used {actual_vehicles_used} vehicles.")


# Print each route
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    route = []
    route_distance = 0
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route.append(node_index)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
    route.append(manager.IndexToNode(index))
    if len(route) > 2:
        print(f"Route for vehicle {vehicle_id}: {route}")
        print(f"Distance of the route: {route_distance}m")

Used 8 vehicles.
Route for vehicle 0: [0, 40, 19, 61, 62, 69, 42, 18, 41, 0]
Distance of the route: 14474m
Route for vehicle 1: [1, 29, 47, 50, 13, 49, 32, 31, 30, 1]
Distance of the route: 15159m
Route for vehicle 2: [2, 60, 39, 64, 73, 37, 58, 70, 66, 52, 2]
Distance of the route: 29550m
Route for vehicle 3: [3, 71, 67, 38, 24, 34, 53, 33, 14, 51, 3]
Distance of the route: 18127m
Route for vehicle 4: [4, 63, 28, 44, 72, 11, 35, 26, 65, 25, 4]
Distance of the route: 16573m
Route for vehicle 5: [5, 36, 46, 16, 43, 12, 79, 22, 20, 21, 5]
Distance of the route: 17956m
Route for vehicle 8: [8, 17, 57, 55, 56, 10, 54, 68, 45, 15, 8]
Distance of the route: 16821m
Route for vehicle 9: [9, 27, 48, 75, 78, 23, 74, 77, 76, 59, 9]
Distance of the route: 15505m


In [20]:
#verifying that all customers nodes are visited in some routes
visited = set()
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        visited.add(node)
        index = solution.Value(routing.NextVar(index))

# Remove depots from the visited set
visited -= set(data['starts'])

assert len(visited) == len(data['distance_matrix']) - len(data['starts']), \
       f"Some nodes were not visited: {set(range(len(data['distance_matrix']))) - visited}"



import matplotlib.pyplot as plt
import random

def plot_routes(manager, routing, solution, data):
    routes = []
    colors = plt.cm.get_cmap('tab10', data['num_vehicles'])  # Adjust if you have more than 10 vehicles

    plt.figure(figsize=(10, 8))

    # Plot all points
    for idx, location in enumerate(data['locations']):
        plt.plot(location[0], location[1], 'ko', markersize=5)
        plt.text(location[0] + 0.3, location[1] + 0.3, str(idx), fontsize=9)

    # Plot the routes
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        if routing.IsEnd(solution.Value(routing.NextVar(index))):
            continue  # Skip unused vehicle

        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(data['locations'][node_index])
            index = solution.Value(routing.NextVar(index))
        # Add end location
        node_index = manager.IndexToNode(index)
        route.append(data['locations'][node_index])
        routes.append(route)

        # Plot this route
        xs, ys = zip(*route)
        plt.plot(xs, ys, marker='o', color=colors(vehicle_id), label=f'Vehicle {vehicle_id}')

    plt.title("VRP Routes")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


if solution:
    plot_routes(manager, routing, solution, data)
else:
    print("No solution found.")